In [1]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 3,626 B in 3s (1,234 B/s)
Reading package lists... Done


In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-01-11 05:33:28--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  1.18MB/s    in 0.8s    

2023-01-11 05:33:30 (1.18 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

## Load Amazon Vine Data into Spark DataFrame

### Method 1: Extract columns from Amazon Data

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Major_Appliances_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Major_Appliances_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show(20)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16199106|R203HPW78Z7N4K|B0067WNSZY|     633038551|FGGF3032MW Galler...|Major Appliances|          5|            0|          0|   N|                Y|If you need a new...|What a great stov...| 2015-08-31|
|         US|   16374060|R2EAIGVLEALSP3|B002QSXK60|     811766671|Best Hand Clothes...|Major Appliances|          5|    

In [5]:
# Create the vine_table DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R203HPW78Z7N4K|          5|            0|          0|   N|                Y|
|R2EAIGVLEALSP3|          5|            1|          1|   N|                Y|
|R1K1CD73HHLILA|          5|            0|          0|   N|                Y|
|R2KZBMOFRMYOPO|          5|            1|          1|   N|                Y|
| R6BIZOZY6UD01|          5|            0|          0|   N|                Y|
|R1MCXZFNF8E7Y0|          1|            0|          0|   N|                Y|
|R3EMB3E3ODR6BW|          5|            2|          2|   N|                Y|
| RJTONVTTOPJ5S|          5|            0|          0|   N|                Y|
|R21U5QZ2CQECUM|          4|            0|          0|   N|                Y|
| RL2BBC51H89DH|          4|            0|          0|   N|     

### Method 2: Pull existing CSV from S3 Bucket

In [6]:
# Store environmental variable
from getpass import getpass
url = getpass("Enter the link to the S3 Bucket and the data file:")

Enter the link to the S3 Bucket and the data file:··········


In [7]:
from pyspark import SparkFiles
spark.sparkContext.addFile(url)
vine_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("vine_table.csv"), sep=",", header=True, inferSchema=True)
vine_df.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R203HPW78Z7N4K|          5|            0|          0|   N|                Y|
|R2EAIGVLEALSP3|          5|            1|          1|   N|                Y|
|R1K1CD73HHLILA|          5|            0|          0|   N|                Y|
|R2KZBMOFRMYOPO|          5|            1|          1|   N|                Y|
| R6BIZOZY6UD01|          5|            0|          0|   N|                Y|
|R1MCXZFNF8E7Y0|          1|            0|          0|   N|                Y|
|R3EMB3E3ODR6BW|          5|            2|          2|   N|                Y|
| RJTONVTTOPJ5S|          5|            0|          0|   N|                Y|
|R21U5QZ2CQECUM|          4|            0|          0|   N|                Y|
| RL2BBC51H89DH|          4|            0|          0|   N|     

## Filter the Data for usable entries

In [8]:
# Filter the data for reviews that have at least twenty votes
popular_df = vine_df.filter(vine_df["total_votes"] >= 20).orderBy("review_id")
popular_df.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R100212BMGLGI0|          1|           18|         21|   N|                N|
|R1002U1ZIY5BUT|          5|           44|         46|   N|                N|
|R100YVAS0FCQ9C|          5|           36|         37|   N|                N|
|R101W5V3SZ8LFQ|          1|           19|         25|   N|                Y|
|R1028R2IEONQZL|          5|           10|         54|   N|                N|
|R10328L5NKZHYU|          2|           24|         27|   N|                Y|
|R104QJT5QCZKZ8|          1|           38|         50|   N|                N|
|R1055S1327MCXB|          5|           45|         48|   N|                N|
|R1057TGFKBGGHS|          5|           23|         23|   N|                N|
|R1058IT10ZEUI7|          5|           69|         72|   N|     

In [9]:
# Filter the results for reviews where at least half of the votes mark them as helpful
helpful_df = popular_df.filter((popular_df["helpful_votes"] / popular_df["total_votes"]) >= 0.5).orderBy("review_id")
helpful_df.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R100212BMGLGI0|          1|           18|         21|   N|                N|
|R1002U1ZIY5BUT|          5|           44|         46|   N|                N|
|R100YVAS0FCQ9C|          5|           36|         37|   N|                N|
|R101W5V3SZ8LFQ|          1|           19|         25|   N|                Y|
|R10328L5NKZHYU|          2|           24|         27|   N|                Y|
|R104QJT5QCZKZ8|          1|           38|         50|   N|                N|
|R1055S1327MCXB|          5|           45|         48|   N|                N|
|R1057TGFKBGGHS|          5|           23|         23|   N|                N|
|R1058IT10ZEUI7|          5|           69|         72|   N|                N|
|R1058JBPKY772B|          1|           22|         25|   N|     

## Separate the results between Vine (paid) and non-Vine (unpaid)

In [10]:
paid_df = helpful_df.filter(helpful_df["vine"] == "Y").orderBy("review_id")
paid_df.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R15NQCQUNOSI4U|          4|           19|         22|   Y|                N|
|R15VXK8I0FPU0A|          4|           22|         23|   Y|                N|
|R19OUHMKNXU3DC|          4|           76|         78|   Y|                N|
|R1B3OGO36CJKDA|          5|           15|         20|   Y|                Y|
|R1CXC7HG447Y0A|          4|           65|         69|   Y|                N|
|R1IDQ0G1S7J9TN|          5|          178|        196|   Y|                N|
|R1L2R8OHO171YG|          5|           66|         68|   Y|                N|
|R1SYUVX7GZZ469|          4|           46|         51|   Y|                N|
|R20O97P5V9CZ21|          3|           21|         27|   Y|                N|
|R214D7Q3CO5W5J|          5|           51|         52|   Y|     

In [11]:
unpaid_df = helpful_df.filter(helpful_df["vine"] == "N").orderBy("review_id")
unpaid_df.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R100212BMGLGI0|          1|           18|         21|   N|                N|
|R1002U1ZIY5BUT|          5|           44|         46|   N|                N|
|R100YVAS0FCQ9C|          5|           36|         37|   N|                N|
|R101W5V3SZ8LFQ|          1|           19|         25|   N|                Y|
|R10328L5NKZHYU|          2|           24|         27|   N|                Y|
|R104QJT5QCZKZ8|          1|           38|         50|   N|                N|
|R1055S1327MCXB|          5|           45|         48|   N|                N|
|R1057TGFKBGGHS|          5|           23|         23|   N|                N|
|R1058IT10ZEUI7|          5|           69|         72|   N|                N|
|R1058JBPKY772B|          1|           22|         25|   N|     

## Count up the Five-Star reviews

In [13]:
total_count = helpful_df.count()
paid_count = paid_df.count()
unpaid_count = unpaid_df.count()
five_count = helpful_df.filter(helpful_df["star_rating"] == 5).count()
five_paid = paid_df.filter(paid_df["star_rating"] == 5).count()
five_unpaid = unpaid_df.filter(unpaid_df["star_rating"] == 5).count()

star_percent = five_count / total_count * 100
paid_percent = five_paid / paid_count * 100
unpaid_percent = five_unpaid / unpaid_count * 100

# Print the results
print(f"There are {total_count:,} reviews that are considered helpful.")
print(f"{paid_count:,} reviews come from Vine members; and {unpaid_count:,} reviews come from non-Vine members.")
print(f"{five_count:,} are 5-Star reviews, comprising {star_percent:.3f}% of all the helpful reviews.")
print(f"{five_paid:,} Vine reviews have a 5-Star rating, forming {paid_percent:.3f}% of the Vine reviews.")
print(f"{five_unpaid:,} non-Vine reviews have a 5-Star rating, forming {unpaid_percent:.3f}% of the non-Vine reviews.")

There are 4,992 reviews that are considered helpful.
35 reviews come from Vine members; and 4,957 reviews come from non-Vine members.
1,981 are 5-Star reviews, comprising 39.683% of all the helpful reviews.
18 Vine reviews have a 5-Star rating, forming 51.429% of the Vine reviews.
1,963 non-Vine reviews have a 5-Star rating, forming 39.601% of the non-Vine reviews.
